args

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'
# os.environ["WANDB_MODE"] = "offline"
# import BalancedDataParallel
import argparse
import datetime
import re
import time,pickle
import wandb
import torch
import numpy as np
import random
# import wandb
import yaml
import transformers
from transformers import AdamW, get_cosine_schedule_with_warmup
from torch.optim import Adam
from tqdm.auto import tqdm
# from path import Path
from sklearn.metrics import mean_absolute_error, f1_score, accuracy_score
from transformers import RobertaPreTrainedModel
from transformers.models.roberta.modeling_roberta import RobertaEmbeddings, RobertaEncoder, RobertaPooler, RobertaLMHead, RobertaClassificationHead
from transformers.modeling_outputs import BaseModelOutputWithPoolingAndCrossAttentions, MaskedLMOutput, SequenceClassifierOutput
from transformers.utils import logging
import torch.nn as nn
from transformers.modeling_outputs import MaskedLMOutput
from transformers import AutoTokenizer, AutoModelForMaskedLM
import pickle
from torch.utils.data import DataLoader
import torch.nn.functional as F
from torch_geometric.nn import GATConv
from torch_geometric.utils import remove_self_loops, add_self_loops


In [2]:

parser = argparse.ArgumentParser()
parser.add_argument('--learning_rate', default=1e-4, type=float, help='Learning Rate')
parser.add_argument('--warmup_proportion', default=0.1, type=float, help='Set Warmup Proportion')
parser.add_argument('--num_epochs', default=80, type=int, help='Set Number of Epochs')
parser.add_argument('--batch_size', default=1, type=int, help='Set Batch Size')
# parser.add_argument('--max_text_length', default=150, type=int, help='Set Max Text Length')
# parser.add_argument('--max_time', default=30, type=int, help='Set Max Time')
parser.add_argument('--mode', default='train', type=str, choices=['train', 'test'], help='Train Or Test Mode')
parser.add_argument('--gradient_accumulation_step', default=8, type=int) # 8

parser.add_argument('--model_output_dir', default='./output/v10/', type=str)
# parser.add_argument('--acoustic_dim', default=100, type=int)
# parser.add_argument('--visual_dim', default=256, type=int)
# parser.add_argument('--text_dim', default=768, type=int)
# parser.add_argument('--beta_shift', default=1, type=int)
# parser.add_argument('--dropout_prob', default=0.5, type=int)

# parser.add_argument('--num_past_utts', default=1, type=int, help='过去的话语数量')
# parser.add_argument('--num_futu_utts', default=0, type=int, help='未来的话语数量')

# lineConGraph 定义 argparse 参数
parser.add_argument('--in_features', type=int, default=768, help='输入特征的维度')
parser.add_argument('--hidden_features1', type=int, default=256, help='隐藏层特征的维度')
parser.add_argument('--hidden_features2', type=int, default=128, help='隐藏层特征的维度')
parser.add_argument('--out_features', type=int, default=7, help='输出特征的维度') # IEMOCAP 6 MELD 7
parser.add_argument('--num_heads', type=int, default=4, help='GATConv 中的注意力头数') #IEMOCAP 1头好
parser.add_argument('--momentum', type=float, default=0.9, help='SGD优化器的动量参数')
parser.add_argument('--dataset', type=str, default='MELD', choices=['IEMOCAP', 'MELD'], help='Dataset to use (IEMOCAP or MELD)')
parser.add_argument('--gamma', type=int, default=8, help='focalloss的gamma')
parser.add_argument('--n', type=int, default=8, help='loss的n')
parser.add_argument('--beta', type=float, default=5e-8, help='beta')
parser.add_argument('--theta', type=float, default=0.001, help='theta')

args = parser.parse_args([])
if args.dataset == 'IEMOCAP':
    args.out_features = 6
    args.class_num = 6
elif args.dataset == 'MELD':
    args.out_features = 7
    args.class_num = 7

print(args)

Namespace(learning_rate=0.0001, warmup_proportion=0.1, num_epochs=80, batch_size=1, mode='train', gradient_accumulation_step=8, model_output_dir='./output/v10/', in_features=768, hidden_features1=256, hidden_features2=128, out_features=7, num_heads=4, momentum=0.9, dataset='MELD', gamma=8, n=8, beta=5e-08, theta=0.001, class_num=7)


In [3]:
logger = logging.get_logger()
# device = 'cuda' if torch.cuda.is_available() else 'cpu'
device = torch.device("cuda:0")

In [4]:
# import numpy as np

# def attention(Z_T, Z_C, Z_F):
#     # 定义权重函数
#     def weight_function(z):
#         return np.tanh(np.dot(W, z.T) + b)

#     # 计算每个矩阵的权重
#     omega_T = weight_function(Z_T)
#     omega_C = weight_function(Z_C)
#     omega_F = weight_function(Z_F)

#     # 计算softmax权重
#     exp_omega = np.exp(omega_T) + np.exp(omega_C) + np.exp(omega_F)
#     q_T = np.exp(omega_T) / exp_omega
#     q_C = np.exp(omega_C) / exp_omega
#     q_F = np.exp(omega_F) / exp_omega

#     # 融合矩阵
#     Z = q_T * Z_T + q_C * Z_C + q_F * Z_F

#     return Z
# def attention(Z_T, Z_C, Z_F):
#     # 随机初始化权重矩阵和偏置向量，并设置requires_grad=True
#     W = torch.randn((1, Z_T.size(1)), requires_grad=True).to(device)
#     b = torch.randn((1, 1), requires_grad=True).to(device)

#     # 定义权重函数
#     def weight_function(z):
#         return torch.tanh(torch.matmul(W, z.t()) + b)

#     # 计算每个矩阵的权重
#     omega_T = weight_function(Z_T)
#     omega_C = weight_function(Z_C)
#     omega_F = weight_function(Z_F)

#     # 计算softmax权重
#     exp_omega = torch.exp(omega_T) + torch.exp(omega_C) + torch.exp(omega_F)
#     q_T = torch.exp(omega_T) / exp_omega
#     q_C = torch.exp(omega_C) / exp_omega
#     q_F = torch.exp(omega_F) / exp_omega

#     # 融合矩阵
#     Z = torch.matmul(q_T, Z_T) + torch.matmul(q_C, Z_C) + torch.matmul(q_F, Z_F)

#     return Z

# # 定义设备
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# # 定义三个输入张量
# Z_T = torch.randn(5, 3).to(device)  # 假设形状为 (5, 3)
# Z_C = torch.randn(5, 3).to(device)  # 假设形状为 (5, 3)
# Z_F = torch.randn(5, 3).to(device)  # 假设形状为 (5, 3)

# # 使用 attention 函数计算融合张量
# Z = attention(Z_T, Z_C, Z_F)

# print("融合张量的形状:", Z.shape)
# print("融合张量内容:")
# print(Z)

# # 示例数据：假设我们有三个2x2矩阵
# Z_T = np.array([[1, 2], [3, 4]])
# Z_C = np.array([[5, 6], [7, 8]])
# Z_F = np.array([[9, 10], [11, 12]])

# # 注意力参数
# W = np.random.rand(2, 2)
# b = np.random.rand(2)

# # 应用注意力机制
# q_T, q_C, q_F, Z = attention(Z_T, Z_C, Z_F)

# print("Attention weights:")
# print("q_T:", q_T)
# print("q_C:", q_C)
# print("q_F:", q_F)
# print("Fused matrix Z:", Z)

In [5]:
import torch

def attention(Z_T, Z_C, Z_F):
    # 随机初始化权重矩阵和偏置向量，并设置requires_grad=True
    W = torch.randn((1, Z_T.size(1)), requires_grad=True).to(Z_T.device)
    b = torch.randn((1, 1), requires_grad=True).to(Z_T.device)

    # 定义权重函数
    def weight_function(z):
        return torch.tanh(torch.matmul(W, z.t()) + b)

    # 计算每个矩阵的权重
    omega_T = weight_function(Z_T)
    omega_C = weight_function(Z_C)
    omega_F = weight_function(Z_F)

    # 计算softmax权重
    exp_omega = torch.exp(omega_T) + torch.exp(omega_C) + torch.exp(omega_F)
    q_T = torch.exp(omega_T) / exp_omega
    q_C = torch.exp(omega_C) / exp_omega
    q_F = torch.exp(omega_F) / exp_omega

    # 融合矩阵
    Z = q_T.view(-1, 1) * Z_T + q_C.view(-1, 1) * Z_C + q_F.view(-1, 1) * Z_F

    return Z

# 示例测试
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Z_T = torch.randn(5, 3).to(device)
# Z_C = torch.randn(5, 3).to(device)
# Z_F = torch.randn(5, 3).to(device)
# Z = attention(Z_T, Z_C, Z_F)
# print("融合张量的形状:", Z.shape)
# print("融合张量内容:")
# print(Z)


lineConGraph

In [6]:
class GATmodule(nn.Module):
    def __init__(self, in_features, hidden_features1, hidden_features2, out_features, num_heads, dropout = 0):
        super(GATmodule, self).__init__()
        self.gat1 = GATConv(in_features, hidden_features1, num_heads)
        self.gat2 = GATConv(hidden_features1 * num_heads, hidden_features2, num_heads)
        # self.dropout = nn.Dropout(0.2)
        self.fc_out = nn.Linear(hidden_features2 * num_heads, out_features) # 对多头的输出降维

    def forward(self, x, edge_index):
        x = F.relu(self.gat1(x, edge_index))
        # x = F.dropout(x, self.dropout, training = self.training)
        x = F.relu(self.gat2(x, edge_index))
        x = self.fc_out(x)
        return x
    
class Attention(nn.Module):
    def __init__(self, in_size, hidden_size=16):
        super(Attention, self).__init__()

        self.project = nn.Sequential(
            nn.Linear(in_size, hidden_size),
            nn.Tanh(),
            nn.Linear(hidden_size, 1, bias=False)
        )

    def forward(self, z):
        w = self.project(z)
        beta = torch.softmax(w, dim=1)
        return (beta * z).sum(1), beta

In [7]:
class lineConGraph(nn.Module):
    def __init__(self, args):
        super(lineConGraph, self).__init__()
        self.sconv1 = GATmodule(args.in_features, args.hidden_features1,args.hidden_features2, args.out_features, args.num_heads)
        self.sconv2 = GATmodule(args.in_features, args.hidden_features1,args.hidden_features2, args.out_features, args.num_heads)
        self.cconv = GATmodule(args.in_features, args.hidden_features1, args.hidden_features2, args.out_features, args.num_heads)
        self.attention = Attention(args.out_features)
        self.MLP = nn.Sequential(
            nn.Linear(args.out_features, args.class_num),
            nn.LogSoftmax(dim=1)
        )
        # self.dropout = nn.Dropout(0.2)
        # self.fc_out1 = nn.Linear(args.out_features * args.num_heads, args.out_features) # 对多头的输出降维
        # self.fc_out2 = nn.Linear(args.out_features * args.num_heads, args.out_features) # 对多头的输出降维
        # self.fc_out3 = nn.Linear(args.out_features * args.num_heads, args.out_features) # 对多头的输出降维
    def forward(self, x, edge_index_other, edge_idx_speaker,edge_idx_longdis):
        # edge_idx_speaker = torch.cat((edge_index_other,edge_idx_speaker), dim=0)
        emb1 = self.sconv1(x, edge_idx_speaker) # Special_GCN out1 -- sadj structure graph
        com1 = self.cconv(x, edge_index_other)  # Common_GCN out1 -- sadj structure graph
        com2 = self.cconv(x, edge_idx_speaker)  # Common_GCN out2 -- fadj feature graph
        emb2 = self.sconv2(x, edge_idx_longdis) # Special_GCN out2 -- fadj feature graph
        # Xcom = (com1 + com2) / 2
        # print('@@@', emb1.shape,emb2.shape,com1.shape, com2.shape, Xcom.shape)
        ##attention
        # emb = torch.stack([emb1, emb2, Xcom], dim=1)
        # emb, att = self.attention(Xcom)
        # output = self.MLP(emb)
        output = self.MLP(emb1+emb2+com1+com2)
        return output

MELDDataset

In [8]:
class MELDDataset(torch.utils.data.Dataset):
    def __init__(self, train=True, n_classes=7):
        path = './MELD_feature_emoberta.pkl'
        
        if n_classes == 7:
            self.videoIDs, self.videoSpeakers, self.videoLabels, self.videoText,\
            self.videoAudio, self.videoSentence, self.trainVid,\
            self.testVid, _ , self.textFea = pickle.load(open(path, 'rb'))
        '''
        label index mapping = {'neutral': 0, 'surprise': 1, 'fear': 2, 'sadness': 3, 'joy': 4, 'disgust': 5, 'anger':6}
        '''
        self.keys = [x for x in (self.trainVid if train else self.testVid)]
        # print(len(self.keys))
        self.len = len(self.keys)
        # self.len = 1
    def __getitem__(self, index):
        vid = self.keys[index]
        labels = self.videoLabels[vid]
        labels = torch.tensor(labels).to(device)
        sentence_vectors = self.textFea[vid].to(device)
        speaker_vectors = self.videoSpeakers[vid]
        edge_idx_other = self.compute_edge_index_with_past_future(sentence_vectors).to(device)
        edge_idx_speaker = self.build_edge_index_with_same_speaker(speaker_vectors).to(device)
        edge_idx_longdis = self.build_edge_index_with_long_dis(speaker_vectors).to(device)
        # print(edge_idx_other)
        # print(edge_idx_speaker)
        # print(edge_idx_longdis)
        return sentence_vectors, edge_idx_other, labels, edge_idx_speaker, edge_idx_longdis
    
    def compute_edge_index_with_past_future(self,sentence_vectors):
        """
        计算句子向量列表的边索引。

        参数：
            sentence_vectors (torch.Tensor): 形状为 (N, d) 的张量，其中 N 是句子数量，d 是句子向量的维度。

        返回：
            torch.LongTensor: 句子之间的边索引，形状为 (2, num_edges)，其中 num_edges 是边的数量。
        """
        num_sentences = sentence_vectors.size(0)
        edge_index = []

        # 添加每句与前一句之间的边
        for i in range(1, num_sentences):
            edge_index.append([i, i-1])
            edge_index.append([i-1, i])
        # 添加自循环
        for i in range(num_sentences):
            edge_index.append([i, i])


        # # 添加每句与后一句之间的边
        # for i in range(num_sentences - 1):
        #     edge_index.append([i, i+1])
        #     edge_index.append([i+1, i])

        return torch.LongTensor(edge_index).t().contiguous()
    
    def build_edge_index_with_same_speaker(self, speakers):
        edge_index = []
        num_sentences = len(speakers)
        for i, speaker in enumerate(speakers):
            # 添加与自身的边
            edge_index.append([i, i])
            for j in range(i+1, num_sentences):
                if speakers[j] == speaker:  # 与当前说话人相同的句子建立边
                    edge_index.append([i, j])
        return torch.tensor(edge_index).t().contiguous()
    
    def build_edge_index_with_long_dis(self, speakers):
        edge_index = []
        num_sentences = len(speakers)
        for i, speaker in enumerate(speakers):
            # 添加与自身的边
            # edge_index.append([i, i])
            for j in range(i, -1, -1):
                if speakers[j] == speaker:  # 与当前说话人相同的前面一个句子建立边
                    edge_index.append([i, j])
                    break
            for j in range(i+1, num_sentences):
                if speakers[j] == speaker:  # 与当前说话人相同的后边一个句子建立边
                    edge_index.append([i, j])
                    break
            # edge_index = remove_self_loops(edge_index)[0]
            # # 然后，添加自环边，以确保每个节点至少有一个自环
            # edge_index, _ = add_self_loops(edge_index, num_nodes=None)
            # # 最后，再次去除重复边
            # edge_index = torch.unique(edge_index, dim=1)
        return torch.tensor(edge_index).t().contiguous()
    
    def __len__(self):
        return self.len
# trainset  = MELDDataset()
# loader = DataLoader(trainset,batch_size=1)
# for data in loader:
#     print(data)
#     break
#     pass

IEMOCAPDataset

In [9]:
class IEMOCAPDataset(torch.utils.data.Dataset):
    def __init__(self, train=True, n_classes=7):
        path = './IEMOCAP_feature_emoberta.pkl'
        self.videoIDs, self.videoSpeakers, self.videoLabels, self.videoText,\
        self.videoAudio, self.videoVisual, self.videoSentence, self.trainVid,\
        self.testVid, self.textFea = pickle.load(open(path, 'rb'), encoding="latin1")
        '''
        label index mapping = {'hap':0, 'sad':1, 'neu':2, 'ang':3, 'exc':4, 'fru':5}
        '''
        # del self.videoVisual['Ses05F_script02_2']
        # del self.videoAudio['Ses05F_script02_2']
        # del self.videoSpeakers['Ses05F_script02_2']
        # del self.videoLabels['Ses05F_script02_2']
        # self.testVid.remove('Ses05F_script02_2')
        self.keys = [x for x in (self.trainVid if train else self.testVid)]
        self.len = len(self.keys)
        # self.len = 5

    def __getitem__(self, index):
        vid = self.keys[index]
        labels = self.videoLabels[vid]
        labels = torch.tensor(labels).to(device)
        sentence_vectors = self.textFea[vid].to(device)
        speaker_vectors = self.videoSpeakers[vid]
        edge_idx_other = self.compute_edge_index_with_past_future(sentence_vectors).to(device)
        edge_idx_speaker = self.build_edge_index_with_same_speaker(speaker_vectors).to(device)
        edge_idx_longdis = self.build_edge_index_with_long_dis(speaker_vectors).to(device)

        return sentence_vectors, edge_idx_other, labels, edge_idx_speaker,edge_idx_longdis
    
    def compute_edge_index_with_past_future(self,sentence_vectors):
        """
        计算句子向量列表的边索引。

        参数：
            sentence_vectors (torch.Tensor): 形状为 (N, d) 的张量，其中 N 是句子数量，d 是句子向量的维度。

        返回：
            torch.LongTensor: 句子之间的边索引，形状为 (2, num_edges)，其中 num_edges 是边的数量。
        """
        num_sentences = sentence_vectors.size(0)
        edge_index = []

        # 添加每句与前一句之间的边
        for i in range(1, num_sentences):
            edge_index.append([i, i-1])
            edge_index.append([i-1, i])
        # 添加自循环
        for i in range(num_sentences):
            edge_index.append([i, i])


        # # 添加每句与后一句之间的边
        # for i in range(num_sentences - 1):
        #     edge_index.append([i, i+1])
        #     edge_index.append([i+1, i])

        return torch.LongTensor(edge_index).t().contiguous()
    
    def build_edge_index_with_same_speaker(self, speakers):
        edge_index = []
        num_sentences = len(speakers)
        for i, speaker in enumerate(speakers):
            # 添加与自身的边
            edge_index.append([i, i])
            for j in range(i+1, num_sentences):
                if speakers[j] == speaker:  # 与当前说话人相同的句子建立边
                    edge_index.append([i, j])
        return torch.tensor(edge_index).t().contiguous()
    
    def build_edge_index_with_long_dis(self, speakers):
        edge_index = []
        num_sentences = len(speakers)
        for i, speaker in enumerate(speakers):
            # 添加与自身的边
            # edge_index.append([i, i])
            for j in range(i, -1, -1):
                if speakers[j] == speaker:  # 与当前说话人相同的前面一个句子建立边
                    edge_index.append([i, j])
                    break
            for j in range(i+1, num_sentences):
                if speakers[j] == speaker:  # 与当前说话人相同的后边一个句子建立边
                    edge_index.append([i, j])
                    break
            # edge_index = remove_self_loops(edge_index)[0]
            # # 然后，添加自环边，以确保每个节点至少有一个自环
            # edge_index, _ = add_self_loops(edge_index, num_nodes=None)
            # # 最后，再次去除重复边
            # edge_index = torch.unique(edge_index, dim=1)
        return torch.tensor(edge_index).t().contiguous()
    
    
    def __len__(self):
        return self.len
# trainset  = MELDDataset()
# loader = DataLoader(trainset,batch_size=1)
# for data in loader:
#     print(data)
#     break
#     pass

fetch_model_optim_sched

In [10]:
def set_random_seeds(seed):
    print("Seed: {}".format(seed))

    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.enabled = False
    torch.backends.cudnn.deterministic = True

    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

def fetch_model_optim_sched( device, args):
    model = lineConGraph(args)
    # 显卡大于1块时，device_ids选择模型载入数据对应的显卡, 但是多卡有不平衡的问题
    # if torch.cuda.device_count() > 1:
    #     model= nn.DataParallel(model,device_ids=[0,1])
    model.to(device)
        
    # optimizer = AdamW(model.parameters(), lr=args.learning_rate)
    optimizer = Adam(model.parameters(), lr=args.learning_rate)
    # optimizer = torch.optim.SGD(model.parameters(), lr=args.learning_rate, momentum=args.momentum)
    
    scheduler = get_cosine_schedule_with_warmup(
        optimizer,
        num_warmup_steps=args.warmup_proportion * args.num_train_optimization_steps,
        num_training_steps=args.num_train_optimization_steps,
    )

    return model, optimizer, scheduler
# fetch_model_optim_sched(config, device)

In [11]:
from torch.autograd import Variable
class FocalLoss(nn.Module):
    r"""
        This criterion is a implemenation of Focal Loss, which is proposed in 
        Focal Loss for Dense Object Detection.

            Loss(x, class) = - \alpha (1-softmax(x)[class])^gamma \log(softmax(x)[class])

        The losses are averaged across observations for each minibatch.

        Args:
            alpha(1D Tensor, Variable) : the scalar factor for this criterion
            gamma(float, double) : gamma > 0; reduces the relative loss for well-classiﬁed examples (p > .5), 
                                   putting more focus on hard, misclassiﬁed examples
            size_average(bool): By default, the losses are averaged over observations for each minibatch.
                                However, if the field size_average is set to False, the losses are
                                instead summed for each minibatch.


    """
    def __init__(self, class_num, alpha=None, gamma=2, size_average=True):
        super(FocalLoss, self).__init__()
        if alpha is None:
            self.alpha = Variable(torch.ones(class_num, 1))
        else:
            if isinstance(alpha, Variable):
                self.alpha = alpha
            else:
                self.alpha = Variable(alpha)
        self.gamma = gamma
        self.class_num = class_num
        self.size_average = size_average

    def forward(self, inputs, targets):
        N = inputs.size(0)
        C = inputs.size(1)
        P = F.softmax(inputs)

        class_mask = inputs.data.new(N, C).fill_(0)
        class_mask = Variable(class_mask)
        ids = targets.view(-1, 1)
        class_mask.scatter_(1, ids.data, 1.)
        #print(class_mask)


        if inputs.is_cuda and not self.alpha.is_cuda:
            self.alpha = self.alpha.cuda()
        alpha = self.alpha[ids.data.view(-1)]

        probs = (P*class_mask).sum(1).view(-1,1)

        log_p = probs.log()
        #print('probs size= {}'.format(probs.size()))
        #print(probs)

        batch_loss = -alpha*(torch.pow((1-probs), self.gamma))*log_p 
        #print('-----bacth_loss------')
        #print(batch_loss)


        if self.size_average:
            loss = batch_loss.mean()
        else:
            loss = batch_loss.sum()
        return loss

loop

In [12]:
# def loss_dependence(emb1, emb2, dim):
#     R = torch.eye(dim).cuda() - (1/dim) * torch.ones(dim, dim).cuda()
#     K1 = torch.mm(emb1, emb1.t())
#     K2 = torch.mm(emb2, emb2.t())
#     RK1 = torch.mm(R, K1)
#     RK2 = torch.mm(R, K2)
#     HSIC = torch.trace(torch.mm(RK1, RK2))
#     return HSIC
def loss_dependence(emb1, emb2):
    R = torch.eye(emb1.size(0)).cuda() - (1/emb1.size(0)) * torch.ones(emb1.size(0), emb1.size(0)).cuda()
    K1 = torch.mm(emb1, emb1.t())
    K2 = torch.mm(emb2, emb2.t())
    RK1 = torch.mm(R, K1)
    RK2 = torch.mm(R, K2)
    HSIC = torch.trace(torch.mm(RK1, RK2))
    return HSIC
def common_loss(emb1, emb2):
    emb1 = emb1 - torch.mean(emb1, dim=0, keepdim=True)
    emb2 = emb2 - torch.mean(emb2, dim=0, keepdim=True)
    emb1 = torch.nn.functional.normalize(emb1, p=2, dim=1)
    emb2 = torch.nn.functional.normalize(emb2, p=2, dim=1)
    cov1 = torch.matmul(emb1, emb1.t())
    cov2 = torch.matmul(emb2, emb2.t())
    cost = torch.mean((cov1 - cov2)**2)
    return cost
def loop(dataloader, model, optimizer, scheduler, device, args, mode='train'):
    loss_cross_entropy_func = F.cross_entropy
    
    if args.dataset == 'IEMOCAP':
        class_nums = [504, 839, 1324, 933, 742, 1468]
        # class_nums = [135, 229, 379, 168, 295, 358]
    else:
        class_nums = [5180, 1355, 308, 794, 1906, 293, 1262]
        # class_nums = [1256, 281, 50, 208, 402, 68, 345]
    total_samples = sum(class_nums)
    alpha = torch.tensor([total_samples / count for count in class_nums])
    alpha /= alpha.sum()  # 归一化
    loss_focalloss_func = FocalLoss(class_num=args.class_num, alpha=alpha, gamma=args.gamma, size_average=True)

    # loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
    model.train() if mode == 'train' else model.eval()
    loss_sum = 0
    preds = []
    labels = []
    epoch_lr = []  # 记录每轮学习率
    with torch.set_grad_enabled(mode == 'train'):
        for step, batch in enumerate(tqdm(dataloader, disable=True)):
            sentence_vectors, edge_idx_other, label, edge_idx_speaker, edge_idx_longdis = batch
            output = model(sentence_vectors.squeeze(0), edge_idx_other.squeeze(0), edge_idx_speaker.squeeze(0), edge_idx_longdis.squeeze(0))
            label = label.squeeze(0)
            # 损失函数
            # loss_class = loss_cross_entropy_func(output, label, weight=alpha.to(device)) 
            # loss_dep = (loss_dependence(emb1, com1) + loss_dependence(emb2, com2))/2
            # loss_com = common_loss(com1,com2)
            # loss = loss_class + args.beta * loss_dep + args.theta * loss_com
            loss_class = loss_cross_entropy_func(output, label)
            # loss = loss_focalloss_func(output, label)
            if args.gradient_accumulation_step > 1:
                # loss /= label.size(0)
                loss_class /= args.gradient_accumulation_step

            loss_sum += loss_class.item()
            if mode == 'train':
                loss_class.backward()

                if (step + 1) % args.gradient_accumulation_step == 0:
                    optimizer.step()
                    scheduler.step()
                    optimizer.zero_grad()
                    current_lr = scheduler.get_last_lr()[0]
                    epoch_lr.append(current_lr)
            preds.extend(torch.argmax(output.detach(), 1).cpu().numpy().tolist())
            labels.extend(label.cpu().numpy().tolist())
        # print("@@@@@@@@@@@",preds, labels)
    return loss_sum / len(dataloader), preds, labels, epoch_lr

# def compute_metrics(pred_scores, gt_sentiment_scores):
#     pred_scores = pred_scores.flatten().detach().numpy()
#     gt_sentiment_scores = gt_sentiment_scores.flatten().detach().numpy()
    
#     mae = mean_absolute_error(y_true=gt_sentiment_scores, y_pred=pred_scores)
#     binary_acc = accuracy_score(y_true=gt_sentiment_scores>0, y_pred=pred_scores>0)
#     f1 = f1_score(y_true=gt_sentiment_scores>0, y_pred=pred_scores>0)
#     corr = np.corrcoef(pred_scores, gt_sentiment_scores)[0][1]
    
#     return mae, binary_acc, f1, corr


main

In [13]:


random_seed = random.randint(1, 1000)
set_random_seeds(random_seed)
random_int = random.randint(1, 1000) # 模型名字不至于重复
print("随机种子:", random_seed, '随机数', random_int)

##Create Output Directory if it doesn't exists
# config['model_internal_dir'] = datetime.datetime.now().strftime('%m%d%Y_%H%M%S')
# if not os.path.exists(config['model_output_dir']):
#     os.mkdir(config['model_output_dir'])

# os.mkdir(Path(config['model_output_dir']) / config['model_internal_dir'])
# 
# print("Model Dir: ", Path(config['model_output_dir']) / config['model_internal_dir'])

# Create All Data Loaders
print("Preparing Datasets and DataLoaders")
if args.dataset == 'MELD':
    train_dataset = MELDDataset(train = True)
    test_dataset = MELDDataset(train = False)
elif args.dataset == 'IEMOCAP':
    train_dataset = IEMOCAPDataset(train = True)
    test_dataset = IEMOCAPDataset(train = False)
    
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=args.batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=args.batch_size)
print("Datasets and DataLoaders Done! :D")

args.num_train_optimization_steps = (int(len(train_dataset) / args.batch_size / args.gradient_accumulation_step) * args.num_epochs)

print("Creating Model")
model, optimizer, scheduler = fetch_model_optim_sched(device,args)
print("Model Created! :D")


Seed: 881
随机种子: 881 随机数 686
Preparing Datasets and DataLoaders
Datasets and DataLoaders Done! :D
Creating Model
Model Created! :D


开始训练

In [14]:
from sklearn.metrics import classification_report, f1_score
import warnings
warnings.filterwarnings("ignore")

print("Starting Training")
print("随机种子:", random_seed, '随机数', random_int)
print('参数：', args)
if args.dataset == 'IEMOCAP':
    target_names = ['hap', 'sad', 'neu', 'ang', 'exc', 'fru']
else:
    target_names = ['neutral', 'surprise', 'fear', 'sadness', 'joy', 'disgust', 'anger']


best_val_loss = None
# wandb.init(config=args,\
#            project='5-27 roberta_fine_tune_SIMCSE_IEOCOMP_锁定.ipynb',\
#             notes='TEOCAMP结果')

for epoch in range(args.num_epochs):
    print(f"Epoch: {epoch}")
    # 打印学习率
    # 获取每个参数组的学习率
    learning_rates = []
    for param_group in optimizer.param_groups:
        learning_rates.append(param_group['lr'])

    # 打印学习率
    for i, lr in enumerate(learning_rates):
        print("Learning rate for parameter group {}: {}".format(i, lr))
        
    tloss, train_epoch_preds, train_epoch_labels, epoch_lr = loop(train_loader, model, optimizer, scheduler, device, args, mode='train')
    testloss, test_epoch_preds,test_epoch_labels, epoch_lr= loop(test_loader, model, optimizer, scheduler, device, args, mode='val')

    testf1 = f1_score(test_epoch_labels,test_epoch_preds ,average='weighted')
    trainf1 = f1_score(train_epoch_labels,train_epoch_preds ,average='weighted')
    # print('学习率',epoch_lr)
    if best_val_loss is None:
        best_val_loss = testloss
    elif best_val_loss > testloss:
        best_val_loss = testloss
        print('testloss loss 下降了↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓')
        
        # 构造模型名称
        model_name = f"{args.dataset}_seed_{random_seed}_int_{random_int}_head_{args.num_heads}"

        # 拼接路径
        PATH = os.path.join(args.model_output_dir, f"{model_name}.ckt")

        print('正在保存模型',PATH)
        torch.save(model.state_dict(), PATH)
        print('--------------------train--------------')
     
        print(classification_report(train_epoch_labels,train_epoch_preds,digits=4, target_names=target_names))
        print('--------------------test--------------')
        print(classification_report(test_epoch_labels,test_epoch_preds,digits=4, target_names=target_names))
    
    # wandb.log({'train_loss':tloss, 'test_loss': testloss})
    # # wandb.log(classification_report(train_epoch_labels,train_epoch_preds,digits=4,output_dict=True))
    # # wandb.log(classification_report(test_epoch_labels,test_epoch_preds,digits=4,output_dict=True,target_names=['hap', 'sad', 'neu', 'ang', 'exc', 'fru']))
    # wandb.log(classification_report(test_epoch_labels,test_epoch_preds,digits=4,output_dict=True))
    
    print("Train Loss: {:.4f}, Train F1: {:.4f} | | Test Loss: {:.4f}, Test F1: {:.4f}".format(tloss, trainf1, testloss, testf1))

    
print("Finished Training! :D")
# wandb.finish(0)

Starting Training
随机种子: 881 随机数 686
参数： Namespace(learning_rate=0.0001, warmup_proportion=0.1, num_epochs=80, batch_size=1, mode='train', gradient_accumulation_step=8, model_output_dir='./output/v10/', in_features=768, hidden_features1=256, hidden_features2=128, out_features=7, num_heads=4, momentum=0.9, dataset='MELD', gamma=8, n=8, beta=5e-08, theta=0.001, class_num=7, num_train_optimization_steps=11520)
Epoch: 0
Learning rate for parameter group 0: 0.0
Train Loss: 0.1976, Train F1: 0.3448 | | Test Loss: 0.1579, Test F1: 0.4456
Epoch: 1
Learning rate for parameter group 0: 1.25e-05
testloss loss 下降了↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓
正在保存模型 ./output/v10/MELD_seed_881_int_686_head_4.ckt
--------------------train--------------
              precision    recall  f1-score   support

     neutral     0.5736    0.9095    0.7035      5180
    surprise     0.5598    0.3557    0.4350   

KeyboardInterrupt: 

In [ ]:
from sklearn.metrics import classification_report, f1_score
import warnings
warnings.filterwarnings("ignore")

print("Starting Training")
print("随机种子:", random_seed, '随机数', random_int)
print('参数：', args)
if args.dataset == 'IEMOCAP':
    target_names = ['hap', 'sad', 'neu', 'ang', 'exc', 'fru']
else:
    target_names = ['neutral', 'surprise', 'fear', 'sadness', 'joy', 'disgust', 'anger']


best_val_loss = None
# wandb.init(config=args,\
#            project='5-27 roberta_fine_tune_SIMCSE_IEOCOMP_锁定.ipynb',\
#             notes='TEOCAMP结果')

for epoch in range(args.num_epochs):
    print(f"Epoch: {epoch}")
    # 打印学习率
    # 获取每个参数组的学习率
    learning_rates = []
    for param_group in optimizer.param_groups:
        learning_rates.append(param_group['lr'])

    # 打印学习率
    for i, lr in enumerate(learning_rates):
        print("Learning rate for parameter group {}: {}".format(i, lr))
        
    tloss, train_epoch_preds, train_epoch_labels, epoch_lr = loop(train_loader, model, optimizer, scheduler, device, args, mode='train')
    testloss, test_epoch_preds,test_epoch_labels, epoch_lr= loop(test_loader, model, optimizer, scheduler, device, args, mode='val')

    testf1 = f1_score(test_epoch_labels,test_epoch_preds ,average='weighted')
    trainf1 = f1_score(train_epoch_labels,train_epoch_preds ,average='weighted')
    # print('学习率',epoch_lr)
    if best_val_loss is None:
        best_val_loss = testloss
    elif best_val_loss > testloss:
        best_val_loss = testloss
        print('testloss loss 下降了↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓')
        
        # 构造模型名称
        model_name = f"{args.dataset}_seed_{random_seed}_int_{random_int}_head_{args.num_heads}"

        # 拼接路径
        PATH = os.path.join(args.model_output_dir, f"{model_name}.ckt")

        print('正在保存模型',PATH)
        torch.save(model.state_dict(), PATH)
        print('--------------------train--------------')
     
        print(classification_report(train_epoch_labels,train_epoch_preds,digits=4, target_names=target_names))
        print('--------------------test--------------')
        print(classification_report(test_epoch_labels,test_epoch_preds,digits=4, target_names=target_names))
    
    # wandb.log({'train_loss':tloss, 'test_loss': testloss})
    # # wandb.log(classification_report(train_epoch_labels,train_epoch_preds,digits=4,output_dict=True))
    # # wandb.log(classification_report(test_epoch_labels,test_epoch_preds,digits=4,output_dict=True,target_names=['hap', 'sad', 'neu', 'ang', 'exc', 'fru']))
    # wandb.log(classification_report(test_epoch_labels,test_epoch_preds,digits=4,output_dict=True))
    
    print("Train Loss: {:.4f}, Train F1: {:.4f} | | Test Loss: {:.4f}, Test F1: {:.4f}".format(tloss, trainf1, testloss, testf1))

    
print("Finished Training! :D")
# wandb.finish(0)

Starting Training
随机种子: 881 随机数 686
参数： Namespace(learning_rate=0.0001, warmup_proportion=0.1, num_epochs=80, batch_size=1, mode='train', gradient_accumulation_step=8, model_output_dir='./output/v10/', in_features=768, hidden_features1=256, hidden_features2=128, out_features=7, num_heads=4, momentum=0.9, dataset='MELD', gamma=8, n=8, beta=5e-08, theta=0.001, class_num=7, num_train_optimization_steps=11520)
Epoch: 0
Learning rate for parameter group 0: 0.0
Train Loss: 0.1976, Train F1: 0.3448 | | Test Loss: 0.1579, Test F1: 0.4456
Epoch: 1
Learning rate for parameter group 0: 1.25e-05
testloss loss 下降了↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓
正在保存模型 ./output/v10/MELD_seed_881_int_686_head_4.ckt
--------------------train--------------
              precision    recall  f1-score   support

     neutral     0.5736    0.9095    0.7035      5180
    surprise     0.5598    0.3557    0.4350   

KeyboardInterrupt: 